## Notebook to research diabetes onset 


e) Matched Controls Diabetes Onset Curve(Blue 3)

   1. Get all the canary patients who have their CC_DIABETES flag equals 0
   on their registered date.
   
       1. Pick a matching algorithm
   2. Create the macthed population
       2. For each canary patient let their registration month be i
       2. Find N(for example 50)  control patients to match this canary patient from the pmpm 
       patients on month i who have CC_DIABETES = 0 on month i. For example the N patients should have the same age 
       and then be the nearest neighbors according to other features(log costs, cc codes etc).
       2. Set the index date for these pmpm controls patients to be month i.
       2. This forms a matched set of roughly 30 times the number of canary patients and each control has an index date.
   
   3. Compute Onset of Diabetes for month i: 
       3. Get all the matched controls with index date prior to month i.
       3. For Month i we define the total number of Diabetes Onsets(DO) as the number of subjects from e.3 that had
        the flag CC_DIABETES = 1 at Month (i) and CC_DIABETES = 0 at Month (i-1) (Note these patients must be in pmpm on both months).
       3. S is the total number of matched controls included in the pmpm both in month i and month i-1 
       3. For Month i we define the diabetes onset rate(DOR) as:
             DOR =  DO/S*1000

       3. Suggestion: Remove from  S the subjects that  have the flag CC_DIABETES = 1 at month i-1

   

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Load Pmpm Canary data

In [3]:
%%time
canary_p = '../../data/CANARY/'
pmpm_p = '../../temp/CC_PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)

CPU times: user 6.22 s, sys: 7.64 s, total: 13.9 s
Wall time: 13.7 s


### IP-ER

In [4]:
def ip_er(group):
    x = np.arange(len(group))+1
    group['AVG_IP'] = (group.ALLOW_IP > 0).cumsum()/x
    group['AVG_ER'] = (group.ALLOW_ER > 0).cumsum()/x
    return group

def ip_er2(group, time):
    x = min(len(group), time)
    c1 = group.IP_BIN.cumsum()
    c2 = group.ER_BIN.cumsum()
    group.loc[:,'IP_BIN2'] = c1.values-c1.shift(periods = x, fill_value =0).values
    group.loc[:,'ER_BIN2'] = c2.values-c2.shift(periods = x, fill_value =0).values
    group['IP_C'] = c1
    group['ER_C'] = c2
    return group

In [5]:
%%time
pmpm = pmpm.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)
pmpm['IP_BIN'] = (pmpm.ALLOW_IP > 0).astype(int)
pmpm['ER_BIN'] = (pmpm.ALLOW_ER > 0).astype(int)

CPU times: user 7.54 s, sys: 2.85 s, total: 10.4 s
Wall time: 10.4 s


In [6]:
%%time
pmpm = pmpm.groupby('PERS_ID').apply(ip_er2, time = 6).reset_index(drop = True)

CPU times: user 28min 20s, sys: 21.1 s, total: 28min 41s
Wall time: 28min 38s


## Log the cost at source - PMPM

In [7]:
%%time
pmpm.ALLOW_AMT = pmpm.ALLOW_AMT.apply(data_process.log_)
pmpm.ALLOW_IP = pmpm.ALLOW_IP.apply(data_process.log_)
pmpm.ALLOW_ER = pmpm.ALLOW_ER.apply(data_process.log_)

CPU times: user 46.4 s, sys: 343 ms, total: 46.7 s
Wall time: 46.7 s


In [8]:
medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
pmpm['LOB'] = 1
pmpm.loc[medicaid_mask, 'LOB'] = 0
pmpm['MEDICAID_LOB'] = pmpm['LOB']
pmpm = pmpm.drop(['LOB'], axis=1)

In [9]:
pmpm = pmpm[col_seq_cc_all]

## Choose Onset Column Name

In [10]:
onset = "CC_DIABETES"

In [11]:
pmpm.head()

PERS_ID     MYR  AGE_AT_MIDMONTH MBR_GNDR  MEDICAID_LOB  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  201908              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  201909              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  201910              119        M             1        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  201

### Process Canary
1. Drop register dates beyond pmpm max date
2. Transform register date to MYR
3. Keep only ids and MYR

### Get Controls and Treated

In [12]:
%%time
canary2 = process_canary(canary, pmpm.MYR.max())
treated, controls = get_tc(pmpm.copy(), canary2)

CPU times: user 8.56 s, sys: 2.91 s, total: 11.5 s
Wall time: 11.5 s


### Keep Canary Patients from their date of registeration and onwards which did not have CC_DIABETES flag on, on their date of registration

In [13]:
%%time
treated2 = in_canary(treated.copy(), canary2)
treated2 = filter_first_month_diabetes(treated2.copy(), onset=onset)
treated2.drop(columns = ['IN_CANARY'], inplace = True)

CPU times: user 5.34 s, sys: 7.81 ms, total: 5.35 s
Wall time: 5.35 s


### Transform with NN

In [14]:
#!aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Latent_Model_m6 ../../temp/TF_Latent_Model_m6

In [15]:
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')

model = models.load_model(path+"/TF_Latent_Model_m6", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [16]:
cols_to_keep = ["PERS_ID", "MYR", onset]

In [17]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [18]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[col_seq_cc_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls[col_seq_cc_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 1min 45s, sys: 25.4 s, total: 2min 11s
Wall time: 1min 18s


### For each patient in the treated group in its registartion month find a population of matched controls in the same month by using 

In [19]:
%%time

columns_to_match = latent_cols 

propensity_func_args = {'match_on':columns_to_match, 'neighbors_per_treated': 5}
matched0, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           propensity_func, 
                                           propensity_func_args)

Mathcing Function: propensity_func


Propensity Score Matching...fitting model
Creating Matched Set...


1592it [31:48,  1.20s/it]


CPU times: user 32min 48s, sys: 5min 30s, total: 38min 18s
Wall time: 33min 34s


In [20]:
random_matching_func_args = {'neighbors_per_treated': 100}
random_controls = matched_population(tr2.copy(), 
                                       cntrls.copy(),
                                       random_matching_func, 
                                       random_matching_func_args)

Mathcing Function: random_matching_func


In [21]:
matched_data0.to_pickle(path+'/ldata_prop/match0_l.p')
random_controls.to_pickle(path+'/ldata_prop/random_l.p')
tr2.to_pickle(path+'/ldata_prop/treated0_l.p')

## Transform Back to Original Space - match0, match1, random_controls
matched0 (latent) -> transform using controls -> matched0_orig <br/> 
matched1 (latent) -> transform using controls -> matched1_orig <br/>
random_controls (latent) -> transform using controls -> random_controls_orig <br/>
treated0(original) -> done -> tr0 (latent) <br/>
treated1(original) -> done -> tr1 (latent) <br/>

In [22]:
matched_data0 = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(matched_data0.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])
random_controls = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(random_controls.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])

In [23]:
matched_data0.to_pickle(path+'/odata_prop/match0_o.p')
random_controls.to_pickle(path+'/odata_prop/random_o.p')
treated2.to_pickle(path+'/odata_prop/treated0_o.p')

### Save to AWS

In [24]:
import os
l_files_list = os.listdir(git_path+"/temp/ldata_prop/")
o_files_list = os.listdir(git_path+"/temp/odata_prop/")

#save in S3
import boto3
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'
for file in l_files_list:
    s3.upload_file( git_path+"/temp/ldata_prop/"+file,
                    bucket, 
                    'rpi/nneehal/DATA/ldata/m6_propens_202108/'+file,
                    ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})
for file in o_files_list:
    s3.upload_file( git_path+"/temp/odata_prop/"+file,
                    bucket, 
                    'rpi/nneehal/DATA/odata/m6_propens_202108/'+file,
                    ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})

## Check Pseudo R2 metric

In [25]:
t_df = tr2.copy()
c_df = cntrls.copy()

In [26]:
t_df["Treated"]=1
c_df["Treated"]=0

In [27]:
df_before_match = pd.concat([t_df, c_df], axis=0)

In [28]:
import pickle
with open(path+'/ldata_prop/match0_l.p', 'rb') as fp:
    m_df = pickle.load(fp)

In [29]:
m_df["Treated"]=0

In [30]:
df_after_match = pd.concat([t_df, m_df], axis=0)

In [31]:
df_before_match[latent_cols]

x1         x2         x3        x4        x5         x6         x7        x8         x9        x10        x11       x12        x13        x14        x15       x16
0       -1.754546  37.688942  13.649632 -9.604260  4.132278  11.206638 -18.391153 -6.095830 -40.455509  13.401694  28.045082 -6.813953 -32.200100 -24.521725 -22.343605  7.942360
1       -1.756822  39.681099  13.957661 -9.884310  4.056077  11.830442 -19.361811 -6.968046 -42.780231  13.623878  29.246895 -7.019114 -34.022720 -26.217098 -23.603809  8.760550
2       -1.756514  39.530651  13.934491 -9.864526  4.062315  11.784522 -19.291286 -6.900780 -42.603687  13.607330  29.160324 -7.000482 -33.887543 -26.088394 -23.506742  8.696609
3       -1.776568  38.578411  13.915503 -9.800701  4.201732  11.455282 -18.823200 -6.280589 -41.433147  13.663046  28.697935 -6.931261 -32.962658 -25.150307 -22.865484  8.173810
4       -1.776828  39.893410  14.084382 -9.967234  4.115385  11.891811 -19.470137 -6.934317 -42.985500  13.758784  29.447453 -7.070744 -34.194515 -26.306126 -23.718779  8.753699
...           ...        ...        ...       ...       ...        ...        ...       ...        ...        ...        ...       ...        ...        ...        ...       ...
9582188 -0.764723  16.488075   5.923328 -4.199619  1.804957   4.909585  -8.064629 -2.693831 -17.645512   5.765053  12.177014 -2.963763 -14.115455 -10.779743  -9.701198  3.531354
9582189 -0.731660  15.003881   5.450493 -3.876187  1.658367   4.483602  -7.342570 -2.411742 -16.026438   5.313735  11.080173 -2.730746 -12.806154  -9.749286  -8.811335  3.152567
9582190 -1.211305  27.554102   9.647993 -6.850790  2.804294   8.247763 -13.478017 -4.892826 -29.680511   9.370453  20.256626 -4.859979 -23.698219 -18.253773 -16.362192  6.132259
9582191 -1.023288  21.315182   7.757983 -5.482162  2.362723   6.352221 -10.411808 -3.408148 -22.813679   7.594094  15.808077 -3.884334 -18.190563 -13.835017 -12.576702  4.464029
9582192 -0.794293  16.356581   5.945467 -4.220518  1.809545   4.883984  -8.000143 -2.624929 -17.481064   5.802963  12.093369 -2.978453 -13.959814 -10.624633  -9.618476  3.433441

[9608161 rows x 16 columns]

In [32]:
def efron_rsquare(y, y_pred):
    n = float(len(y))
    t1 = np.sum(np.power(y-y_pred, 2.0))
    t2 = np.sum(np.power((y-np.sum(y)/n), 2.0))
    return 1.0-(t1/t2)

In [33]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=False, solver='lbfgs').fit(df_before_match[latent_cols].values, df_before_match.Treated.values)

y = df_after_match.Treated.values
y_pred = clf.predict_proba(df_after_match[latent_cols].values)[:,1]

In [34]:
efron_rsquare(y, y_pred)

In [35]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=False, solver='lbfgs').fit(df_after_match[latent_cols].values, df_after_match.Treated.values)

y = df_after_match.Treated.values
y_pred = clf.predict_proba(df_after_match[latent_cols].values)[:,1]

In [36]:
efron_rsquare(y, y_pred)

### Run the Diabetes onset Function

In [37]:
%%time
con_onset0 = diabet_onset(matched_data0.copy(), 'Diabetes_Con0')

treated_onset0 = diabet_onset(treated2.copy(), 'Diabetes_Tr')

pmpm2 = filter_first_month_diabetes(pmpm.copy())
pmpm_onset = diabet_onset(pmpm2, 'Diabetes_PM')

CPU times: user 4min 34s, sys: 13.4 s, total: 4min 48s
Wall time: 4min 45s


In [38]:
random_onset = diabet_onset(random_controls.copy(), 'Diabetes_Rnd')

In [39]:
con_onset20 = con_onset0.rename(columns = {'Sample': 'Sample_Con0', 'Per_1000':'Per_1000_Con0'})

treated_onset20 = treated_onset0.rename(columns = {'Sample': 'Sample_Tr0', 'Per_1000':'Per_1000_Tr0'})

pmpm_onset2 = pmpm_onset.rename(columns = {'Sample': 'Sample_PM', 'Per_1000':'Per_1000_PM'})

In [40]:
random_onset2 = random_onset.rename(columns = {'Sample': 'Sample_PM', 'Per_1000':'Per_1000_PM'})
joint = pd.concat((con_onset20, treated_onset20,  pmpm_onset2, random_onset2), axis = 1)

### Diabetes Onset Per Month

1. **_Con : For Matched Controls Population**
2.  **_Tr : For Canary Treated Population**
3. **_PM : For the whole PMPM Population**
4. **_Rnd: For a Random Matching**

In [41]:
joint.describe().loc[['mean', 'std']]

Sample_Con0  Diabetes_Con0  Per_1000_Con0  Sample_Tr0  Diabetes_Tr  Per_1000_Tr0      Sample_PM  Diabetes_PM  Per_1000_PM     Sample_PM  Diabetes_Rnd  Per_1000_PM
mean  2379.800000       4.422222       1.593456  537.422222     1.155556      1.526206  147321.277778   288.907407     1.963740  36050.266667     62.266667     1.708601
std   2148.765402       4.683088       1.419932  492.239848     1.491728      2.414896   15040.213357    49.399514     0.279397  30113.864457     51.433275     0.832538

In [42]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-dark')

In [43]:
joint2 = pd.concat((con_onset0,  treated_onset0,   pmpm_onset, random_onset), axis = 1)
jointcon0 = joint2.Per_1000.iloc[1:, 0].reset_index()
jointTr0 = joint2.Per_1000.iloc[1:, 1].reset_index()
jointpmpm = joint2.Per_1000.iloc[1:, 2].reset_index()
jointrandom = joint2.Per_1000.iloc[1:, 3].reset_index()

**Plot Top Left**: $f(x)$ = Onset Per 1000 per month </br>
**Plot Top Right and Bottom Left**: $F(x) = \int_0^x f(y)dy$ Cumulative sum of the rates

In [44]:
from diabetes_onset_plotter import plot_onset, plot_onset2
names = ['Matched Controls', 'Treated', 'Random']
f, ax = plot_onset2(jointcon0, jointTr0, jointrandom, names = names)
#f.savefig('figs/onset.png')
plt.show()

ImportError: cannot import name 'plot_onset'

In [ ]:
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

#model 4
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR', 'CC_CHRONIC_KIDNEY', 'CC_HEART_FAILURE', 'CC_ISCHEMIC', 
                    'CC_STROKE', 'CC_HYPERLIPIDEMIA', 'CC_HYPERTENSION']

tr_desc0 = treated2[columns_to_match].describe().loc[['mean', 'std']]
tr_desc0 = tr_desc0.T

con_desc0 = matched_data0[columns_to_match].describe().loc[['mean', 'std']]
con_desc0 = con_desc0.T

pmpm_desc = pmpm2[columns_to_match].describe().loc[['mean', 'std']]
pmpm_desc = pmpm_desc.T

random_desc = random_controls[columns_to_match].describe().loc[['mean', 'std']]
random_desc = random_desc.T

alli = pd.concat((tr_desc0, con_desc0, pmpm_desc, random_desc), axis = 1, keys = ['Treated',
                                                                                 'Matched Controls',
                                                                                'Genaral Population', 
                                                                                 'Random'] )

In [ ]:
alli

## Distribution of latent space

In [ ]:
import pickle
with open(path+'/ldata_prop/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_prop/random_l.p', 'rb') as fp:
    random_latent = pickle.load(fp)
with open(path+'/ldata_prop/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)

In [ ]:
columns_to_match = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

tr_desc0 = treated_sick_latent[columns_to_match].describe().loc[['mean','std']].T
con_desc0 = controls_sick_latent[columns_to_match].describe().loc[['mean','std']].T
random_desc = random_latent[columns_to_match].describe().loc[['mean','std']].T

alli_latent = pd.concat((tr_desc0, con_desc0, random_desc), axis = 1, keys = ['Treated', 'Matched Controls', 'Random'] )

In [ ]:
alli_latent